##Load Dataset

- torch = Main PyTorch library
- torchvision = Useful for image datasets and models

In [ ]:
import torch
import torchvision

In [3]:
# Load Manual
from google.colab import files
uploaded = files.upload()
print(uploaded)

Saving Marks BIO 3105 D Section.xlsx to Marks BIO 3105 D Section.xlsx
{'Marks BIO 3105 D Section.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00A7\x82\xcfr\x01\x00\x00\x04\x05\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [4]:
#Mount google drive
from google.colab import drive
path ='/content/drive/MyDrive/Colab Notebooks/BREAST CANCER - Dataset.xlsx'
print(path)

/content/drive/MyDrive/Colab Notebooks/BREAST CANCER - Dataset.xlsx


In [5]:
#Download from URl
import kagglehub
path = kagglehub.dataset_download("gpiosenka/cards-image-datasetclassification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cards-image-datasetclassification


## If it is an Image Dataset (folder of images)

- We will use ImageFolder:
- Folder Structure Example:
  your_dataset/
    
    class1/
        image1.jpg
        image2.jpg
    class2/
        image3.jpg
        image4.jpg


- transforms = Preprocessing images (resize, normalize, convert to tensor)
- batch_size=32 = 32 images at once
- shuffle=True = Mix images randomly (good for training)


In [8]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)), #Resize
    transforms.ToTensor(), #tensor
])

# Load dataset
dataset = datasets.ImageFolder(root=path, transform=transform)

# Create dataloader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

##If it is a CSV Dataset (like MNIST labels)

- Example: data.csv
- You can use pandas + torch:

In [12]:
import pandas as pd                 # pandas is used for loading and handling CSV data
import torch
from torch.utils.data import Dataset, DataLoader   # Import Dataset and DataLoader for managing datasets

# Load the CSV file using pandas
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/annual-enterprise-survey-2023-financial-year-provisional.csv')

# Create a custom Dataset class for handling our CSV data
class MyDataset(Dataset):
    def __init__(self, dataframe):   # Constructor; takes a pandas dataframe
        self.data = dataframe         # Store the dataframe inside the class

    def __len__(self):                # Returns the total number of samples
        return len(self.data)

    def __getitem__(self, idx):        # Fetch one sample at index `idx`
        row = self.data.iloc[idx]      # Get one row of data
        features = torch.tensor(row[:-1].values, dtype=torch.float32)  # Features (all columns except the last)
        label = torch.tensor(row[-1], dtype=torch.long)                # Label (last column)
        return features, label         # Return (features, label) as a pair


# Create a dataset object
dataset = MyDataset(data)             # Pass the loaded CSV data to our custom dataset

# Create a dataloader object
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

##Check If Dataset is Loading Correctly

-  If you see output like [32, 3, 224, 224] (for images) → it means batch size 32, 3 color channels (RGB), 224x224 size

- If you see [32] for labels, it means 32 labels (one for each sample).



In [14]:
# Check a single batch from the dataloader
for features, labels in dataloader:   # Loop over batches
    print(features.shape)             # Print the shape of feature batch (e.g., torch.Size([32, 10]) )
    print(labels.shape)               # Print the shape of label batch (e.g., torch.Size([32]))
    break

In [18]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

# Load the CSV file using pandas
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/annual-enterprise-survey-2023-financial-year-provisional.csv')

# Create a custom Dataset class for handling our CSV data
class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        # Convert features to numeric, handling non-numeric values
        features = pd.to_numeric(row[:-1], errors='coerce').fillna(0).values # Convert to numeric, replace errors with NaN, then fill NaN with 0
        features = torch.tensor(features, dtype=torch.float32)  # Convert to tensor
        # Convert label to numeric if it's not already
        try:
            label = torch.tensor(int(row[-1]), dtype=torch.long)  # Try converting to int directly
        except ValueError:
            # Handle the case where conversion to int fails (e.g., string labels)
            # You might need to create a mapping from string labels to numerical labels
            # For example:
            label_mapping = {'label_1': 0, 'label_2': 1, 'label_3':2}  # Replace with your actual labels
            label = torch.tensor(label_mapping.get(row[-1], -1), dtype=torch.long)  # Get numerical label from mapping, default to -1 if not found

        return features, label

# Create a dataset object
dataset = MyDataset(data)

# Create a dataloader object
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for features, labels in dataloader:   # Loop over batches
    print(features.shape)             # Print the shape of feature batch (e.g., torch.Size([32, 10]) )
    print(labels.shape)               # Print the shape of label batch (e.g., torch.Size([32]))
    break

torch.Size([32, 9])
torch.Size([32])


<ipython-input-18-4217863fa846>:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = torch.tensor(int(row[-1]), dtype=torch.long)  # Try converting to int directly
<ipython-input-18-4217863fa846>:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = torch.tensor(label_mapping.get(row[-1], -1), dtype=torch.long)  # Get numerical label from mapping, default to -1 if not found


##Training Loop (How to Use This Data for Training)

In [20]:
for epoch in range(epochs):                # Loop over number of epochs
    for features, labels in dataloader:    # Loop over each batch of data

        # Forward pass: pass features into the model
        outputs = model(features)

        # Compute the loss between outputs and true labels
        loss = criterion(outputs, labels)

        # Backward pass: calculate gradients
        optimizer.zero_grad()              # Reset gradients to zero
        loss.backward()                    # Backpropagation (calculate gradients)
        optimizer.step()                   # Update model parameters